# Load and Clean the Data

In [1]:
%run ./preprocessing.ipynb

E    52268
K    13372
Name: customer_gender, dtype: int64


In [2]:
df_male.shape, df_female.shape

((52268, 42), (13372, 42))

# Self-Supervised Similarity Measure

In [5]:
%run ./similarity.ipynb

E    52268
K    13372
Name: customer_gender, dtype: int64


2022-05-09 11:33:17.119033: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-05-09 11:33:17.119060: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-05-09 11:33:17.119330: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
